# Imports

In [12]:
import random
import copy

# Data Loading

In [32]:
# building master deck
deck = []
for i in range(13):
    if i > 1:
        for j in range(i):
            deck.append(str(i))
    else:
        deck.append(str(i))
for card in ["sc", "f3", "fr"]:
    for _ in range(3):
        deck.append(card)
for i in range(2, 11, 2):
    deck.append(f"+{i}")
deck.append("x2")

In [173]:
def pop_from_deck(ls, deck):
    for item in ls:
        deck.remove(str(item))
    return deck

curr_deck = deck.copy()
drawn = [2, 10, 1, 3, 8]
pop_from_deck(drawn, curr_deck)

['0',
 '2',
 '3',
 '3',
 '4',
 '4',
 '4',
 '4',
 '5',
 '5',
 '5',
 '5',
 '5',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '7',
 '7',
 '7',
 '7',
 '7',
 '7',
 '7',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 'sc',
 'sc',
 'sc',
 'f3',
 'f3',
 'f3',
 'fr',
 'fr',
 'fr',
 '+2',
 '+4',
 '+6',
 '+8',
 '+10',
 'x2']

In [174]:
def calc_score(drawn):
    relevant_nums = [item for item in drawn if item not in ["sc", "f3", "fr"]]
    
    flip_seven = []
    sum = 0
    mod = 0
    double = 1
    flipped = 0

    for item in relevant_nums:
        if "x" in item:
            double = 2
        elif "+" in item:
            mod += int(item.strip("+"))
        else:
            if item in flip_seven:
                return 0
            else:
                flip_seven.append(item)
            sum += int(item)
    if len(flip_seven) == 7:
        flipped = 1
    final = (sum * double) + mod + (flipped * 15)
    return final

In [175]:
def check_bust(drawn, deck):
    drawn = [str(item) for item in drawn]
    
    # current score
    curr_score = calc_score(drawn)
    print(f"Current Score: {curr_score}")
    print("")

    # counting cards left
    cards_left = len(deck)
    deck_counter = {}
    for item in deck:
        if item not in deck_counter.keys():
            deck_counter[item] = 1
        else:
            deck_counter[item] += 1

    # finding expected value
    expected_values_data = []
    total_expected_value = 0
    for k, v in deck_counter.items():
        temp = drawn.copy()
        temp.append(k)
        temp_perc = v/cards_left
        temp_total = calc_score(temp)
        delta = temp_total - curr_score
        expected_value = temp_perc * delta
        total_expected_value += expected_value
        expected_values_data.append((k, temp_perc, temp_total, delta, expected_value))
    expected_values_data = sorted(expected_values_data, key=lambda x: x[4], reverse=True)
    if total_expected_value > 0:
        print("HIT")
    else:
        print("STAY")
    print(f"Expected Value: {total_expected_value:.2f}")
    for tuple_set in expected_values_data:
        tag = f"+{tuple_set[3]}" if tuple_set[3] > 0 else str(tuple_set[3])
        print(f"   {tuple_set[0]:>3} ({tuple_set[1]*100:5.2f}%) -> {tuple_set[2]:>3} ({tag:>3}) | {tuple_set[4]:>6.2f}")
    print("")

    # checking busts
    bustable = []
    bust_total = 0
    for item in drawn:
        if item not in ["sc", "f3", "fr"]:
            try:
                perc = deck_counter[item]/cards_left
                bustable.append((item, perc))
                bust_total += perc
            except:
                pass
        else:
            pass
    bustable = sorted(bustable, key=lambda x: x[1], reverse=True)
    print(f"Bust Chance: {bust_total * 100:.2f}%")
    for bust_num in bustable:
        print(f"   {bust_num[0]:>3} ({bust_num[1]*100:5.2f}%)")
    print("")

    # checking events
    events = []
    event_total = 0
    for item in ["sc", "f3", "fr"]:
        try:
            perc = deck_counter[item]/cards_left
            events.append((item, perc))
            event_total += perc
        except:
            continue
    print(f"Event Chance: {event_total*100:.2f}%")
    for bust_num in events:
        print(f"   {bust_num[0]:>3} ({bust_num[1]*100:5.2f}%)")
    
check_bust(drawn, curr_deck)

Current Score: 24

HIT
Expected Value: 0.79
    12 (13.48%) ->  36 (+12) |   1.62
    11 (12.36%) ->  35 (+11) |   1.36
     9 (10.11%) ->  33 ( +9) |   0.91
     7 ( 7.87%) ->  31 ( +7) |   0.55
     6 ( 6.74%) ->  30 ( +6) |   0.40
     5 ( 5.62%) ->  29 ( +5) |   0.28
    x2 ( 1.12%) ->  48 (+24) |   0.27
     4 ( 4.49%) ->  28 ( +4) |   0.18
   +10 ( 1.12%) ->  34 (+10) |   0.11
    +8 ( 1.12%) ->  32 ( +8) |   0.09
    +6 ( 1.12%) ->  30 ( +6) |   0.07
    +4 ( 1.12%) ->  28 ( +4) |   0.04
    +2 ( 1.12%) ->  26 ( +2) |   0.02
     0 ( 1.12%) ->  24 (  0) |   0.00
    sc ( 3.37%) ->  24 (  0) |   0.00
    f3 ( 3.37%) ->  24 (  0) |   0.00
    fr ( 3.37%) ->  24 (  0) |   0.00
     2 ( 1.12%) ->   0 (-24) |  -0.27
     3 ( 2.25%) ->   0 (-24) |  -0.54
     8 ( 7.87%) ->   0 (-24) |  -1.89
    10 (10.11%) ->   0 (-24) |  -2.43

Bust Chance: 21.35%
    10 (10.11%)
     8 ( 7.87%)
     3 ( 2.25%)
     2 ( 1.12%)

Event Chance: 10.11%
    sc ( 3.37%)
    f3 ( 3.37%)
    fr ( 3.37%)


['0',
 '2',
 '3',
 '3',
 '3',
 '4',
 '4',
 '4',
 '4',
 '5',
 '5',
 '5',
 '5',
 '5',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '7',
 '7',
 '7',
 '7',
 '7',
 '7',
 '7',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 'sc',
 'sc',
 'f3',
 'f3',
 'f3',
 'fr',
 'fr',
 'fr',
 '+2',
 '+4',
 '+6',
 '+8',
 '+10']